#### Setup

In [1]:
import sys
import numpy as np
import pandas as pd
import math
import random
from concorde.tsp import TSPSolver
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
print(sys.version)


3.7.6 | packaged by conda-forge | (default, Mar 23 2020, 22:45:16) 
[Clang 9.0.1 ]


In [2]:
# GLOBAL VARIABLES
field_width = 100
field_height = 100
depot_x = 50
depot_y = 50

In [3]:
class Instance():
    
    def __init__(self, xlocs, ylocs, demands, solve_TSP=True):

        self.size = len(demands)-1
        self.demands = demands
        self.xlocs = xlocs
        self.ylocs = ylocs
        self.distances = self.calc_distance_matrix()
        self.optimal_routes = 'None'
        self.tour = 'None'
        if solve_TSP:
            self.tour = self.solve_TSP()
        
    def calc_distance_matrix(self):

        distances = np.zeros((self.size+1, self.size+1), dtype=float)
        for i in range(self.size+1):
            for j in range(self.size+1):
                new_dist = math.sqrt((self.xlocs[i]-self.xlocs[j])**2 + (self.ylocs[i]-self.ylocs[j])**2)
                distances[i,j] = new_dist
        return distances

    def get_lowerbound(self, capacity):
        return (2/capacity) * sum([self.demands[i]*self.distances[0,i]
                                        for i in range(len(self.demands))])
    
    def get_fleet_size(self, route_size):
        assert self.size % route_size == 0, "Number of customers must be evenly divisible by the route size."
        return int(self.size / route_size)
    
    def solve_TSP(self):
        solver = TSPSolver.from_data(self.xlocs, self.ylocs, 'EUC_2D')
        solution = solver.solve()
        return list(solution.tour)
    
    def save_optimal_routes(self, route_list):
        self.optimal_routes = route_list

In [9]:
def get_trip_count(route_list):
    
    assert type(route_list[0]) == list, "route_list must be a list of lists (routes)"
    count = 0
    for route in route_list:
        if route != []:
            count += 1
    return count
    
def get_circular_cost(inst,segment):
    if len(segment) == 0:
        return 0
    else:
        return sum([inst.distances[segment[i],segment[i+1]] for i in range(len(segment)-1)])

def get_radial_cost(inst,segment):
    """Assumes vehicle travels to/from the depot at segment endpoints."""
    if len(segment) == 0:
        return 0
    else:
        return inst.distances[0,segment[0]] + inst.distances[0,segment[-1]]

def get_total_cost(inst,segment):
    return get_circular_cost(inst,segment)+get_radial_cost(inst,segment)

In [4]:
def solve_VRP(inst, capacity):

    def create_data_model(inst, capacity):
        data = {}
        data['distance_matrix'] = inst.distances
        data['demands'] = inst.demands
        data['vehicle_capacities'] = [capacity]*inst.size
        data['num_vehicles'] = inst.size
        data['depot'] = 0
        return data

    def get_routes(solution, routing, manager):
        """Get vehicle routes from a solution and store them in an array."""
        # Get vehicle routes and store them in a two dimensional array whose
        # i,j entry is the jth location visited by vehicle i along its route.
        routes = []
        for route_nbr in range(routing.vehicles()):
            index = routing.Start(route_nbr)
            route = [manager.IndexToNode(index)]
            while not routing.IsEnd(index):
                index = solution.Value(routing.NextVar(index))
                route.append(manager.IndexToNode(index))
            routes.append(route)
        return routes

    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]
    
    ###################
    ### RUN PROGRAM ###
    ###################
    
    # Zero cost if no demands
    if all(dem == 0 for dem in inst.demands):
        return (0, 0, 0)
    
    # Set up data model
    data = create_data_model(inst, capacity)
    
    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']), data['num_vehicles'], data['depot'])
    
    # Create routing model
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add capacity constraint
    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem
    solution = routing.SolveWithParameters(search_parameters)
    all_routes = get_routes(solution, routing, manager)
    nonempty_routes = [route for route in all_routes if not all(i == 0 for i in route)]
    return nonempty_routes

def solve_SDVRP(inst,capacity):
    
    split_xlocs = [[0]] + [[inst.xlocs[i]]*demands[i] for i in range(1,len(inst.demands))]
    split_ylocs = [[0]] + [[inst.ylocs[i]]*demands[i] for i in range(1,len(inst.demands))]
    split_demands = [[0]] + [[inst.demands[i]]*demands[i] for i in range(1,len(inst.demands))]

    split_xlocs = [v for sublist in split_xlocs for v in sublist]
    split_ylocs = [v for sublist in split_ylocs for v in sublist]
    split_demands = [v for sublist in split_demands for v in sublist]
    split_inst = Instance(split_xlocs, split_ylocs, split_demands)
    return solve_VRP(split_inst,capacity)

In [20]:
def get_dedicated_routes(inst, route_size):
    
    tour =  inst.tour[1:]
    routes = []
    for i in range(0,len(tour),route_size):
        new_route = tour[i:i+route_size]
        routes.append(new_route)
    return routes

def get_overlapped_routes(inst, route_size, overlap_size):
    
    tour = inst.tour[1:]
    routes = []
    for i in range(0,len(tour),route_size):
        new_route = tour[i:i+route_size+overlap_size]
        routes.append(new_route)
    return routes

In [124]:
def create_full_trips(inst, route_list, capacity, route_size=inst.size, overlap_size=inst.size):
    """Note: currently allow truck to make trip even if starts in overlapped section."""
    
    assert type(route_list[0]) == list, "route_list must be a list of lists (routes)"

    segments = []
    all_dict = dict([(inst.tour[i],0) for i in range(1,len(inst.tour)) if inst.demands[inst.tour[i]] !=0])
    for route in  route_list:
        i = 0
        seg_dict = {}
        while i < len(route):
            cust = route[i]
            for d in range(1,inst.demands[cust]+1):
                
                if all_dict[cust] == inst.demands[cust]:
                    # already full served
                    pass
                elif cust not in seg_dict:
                    # begin service
                    seg_dict[cust] = 1
                    all_dict[cust] += 1
                else:
                    # continue service
                    seg_dict[cust] += 1
                    all_dict[cust] += 1
                    
                if sum(seg_dict.values()) == capacity:
                    seg_array = list(seg_dict)
                    segments.append(seg_array)
                    seg_dict = {}
                    if i+1 > route_size:
                        i = len(route) # force to move to next route
                        break

            i+=1
        seg_array = list(seg_dict)
        segments.append(seg_array)
    return segments

In [108]:
N = 10
dmin = 0
dmax = 8
capacity = 20
route_size = 5
overlap_size = 5
cust_x = field_width*np.random.random(N)
cust_y = field_height*np.random.random(N)
cust_dems = list(np.random.randint(dmin,dmax,N))
xlocs = list(np.append([depot_x], cust_x))
ylocs = list(np.append([depot_y], cust_y))
demands = list(np.append([0], cust_dems))
inst = Instance(xlocs, ylocs, demands)

In [125]:
print('Customer count:\t', inst.size)
print('Veh Capacity:\t', capacity)
print('Ded route size:\t', route_size)
print('Overlap size:\t', overlap_size)
print('Big TSP tour:\t', inst.tour[1:])
print('Tour demands:\t', [inst.demands[c] for c in inst.tour[1:]])
print('LB Cost:\t', inst.get_lowerbound(capacity).round(1))

print('\n\n----- DEDICATED -----')
routes = get_dedicated_routes(inst, route_size)
realized = create_full_trips(inst,routes,capacity)
print('\nSkeleton routes:', *routes, sep="\n")
print('\nRealized trips:', *realized, sep="\n")
print('\nTrip count:\t', get_trip_count(realized))
print('Radial cost:\t', sum([get_radial_cost(inst,seg) for seg in realized]).round(1))
print('Circular cost:\t', sum([get_circular_cost(inst,seg) for seg in realized]).round(1))
print('Total cost:\t', sum([get_total_cost(inst,seg) for seg in realized]).round(1))

print('\n\n----- OVERLAPPED -----')
print('Notes:')
print('(1) Currently allows truck to make trip even if starts in overlapped section.')
print('(2) Implementation currently does not follow algorithm from paper')
routes = get_overlapped_routes(inst, route_size, overlap_size)
realized = create_full_trips(inst,routes,capacity,route_size,overlap_size)
print('\nSkeleton routes:', *routes, sep="\n")
print('\nRealized trips:', *realized, sep="\n")
print('\nTrip count:\t', get_trip_count(realized))
print('Radial cost:\t', sum([get_radial_cost(inst,seg) for seg in realized]).round(1))
print('Circular cost:\t', sum([get_circular_cost(inst,seg) for seg in realized]).round(1))
print('Total cost:\t', sum([get_total_cost(inst,seg) for seg in realized]).round(1))

print('\n\n----- Fully Flexible -----')
realized = create_full_trips(inst,[inst.tour],capacity)
print('\nRealized trips:', *realized, sep="\n")
print('\nTrip count:\t', get_trip_count(realized))
print('Radial cost:\t', sum([get_radial_cost(inst,seg) for seg in realized]).round(1))
print('Circular cost:\t', sum([get_circular_cost(inst,seg) for seg in realized]).round(1))
print('Total cost:\t', sum([get_total_cost(inst,seg) for seg in realized]).round(1))

Customer count:	 10
Veh Capacity:	 20
Ded route size:	 5
Overlap size:	 5
Big TSP tour:	 [2, 7, 10, 1, 6, 4, 8, 9, 3, 5]
Tour demands:	 [1, 2, 6, 1, 2, 7, 1, 4, 0, 6]
LB Cost:	 136.6


----- DEDICATED -----

Skeleton routes:
[2, 7, 10, 1, 6]
[4, 8, 9, 3, 5]

Realized trips:
[2, 7, 10, 1, 6]
[4, 8, 9, 5]

Trip count:	 2
Radial cost:	 151.2
Circular cost:	 239.2
Total cost:	 390.3


----- OVERLAPPED -----
Notes:
(1) Currently allows truck to make trip even if starts in overlapped section.
(2) Implementation currently does not follow algorithm from paper

Skeleton routes:
[2, 7, 10, 1, 6, 4, 8, 9, 3, 5]
[4, 8, 9, 3, 5]

Realized trips:
[2, 7, 10, 1, 6, 4, 8]
[]
[9, 5]

Trip count:	 2
Radial cost:	 156.9
Circular cost:	 252.5
Total cost:	 409.4


----- Fully Flexible -----

Realized trips:
[2, 7, 10, 1, 6, 4, 8]
[9, 5]

Trip count:	 2
Radial cost:	 156.9
Circular cost:	 252.5
Total cost:	 409.4
